In [1]:
import antigravity

In [2]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


# Multiprocessing and Multithreading

![Image](https://image.slidesharecdn.com/patternoriented-software-architecture-patterns-for-concurrent-and-networked-objects-3939/95/patternoriented-software-architecture-patterns-for-concurrent-and-networked-objects-78-728.jpg?cb=1195276281)

###### Goal
<ul>
<li>Speed up code by using multiple processes</li>
</ul>

###### Options
<ul>
<li>Multithreading</li>
<li>Multiprocessing</li>
</ul>

#### Multithreading

Can use when
<ul>
<li>Lots time waiting around for a response</li>
<ul>
<li>Network Requests - http get, post, put</li>
</ul>
<li>
Lots of I/O (Read, Write, Send, Recv...)
</li>
</ul>

##### Still Bound by Global Interpreter Lock

## CPU Bound Threading

In [3]:
from __future__ import division
from threading import Thread
import time

Make a list with 10 million 10's.

In [4]:
len_x = 10000000
x = [10]*len_x
x[:15]

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

Squaring Function

In [5]:
def squared(num):
    num**2

In [6]:
def squareList(nums):
    for i in nums:
        squared(i)

### Square all of the numbers in the list.

###### No threading

In [7]:
start = time.time() #  Get current time
squareList(x)
serialprocesstime = time.time() - start
print("Serial computation took {} seconds.".format(serialprocesstime))

Serial computation took 5.55535387993 seconds.


###### With threads

give each thread 1/4th of the work

In [16]:
x2 = [10]*(len_x//4)
t1 = Thread(target=squareList, args=(x2,))
t2 = Thread(target=squareList, args=(x2,))
t3 = Thread(target=squareList, args=(x2,))
t4 = Thread(target=squareList, args=(x2,))

start = time.time()
t1.start()
t2.start()
t3.start()
t4.start()
t1.join()
t2.join()
t3.join()
t4.join()
print("4 threads took {} seconds.".format(time.time() - start))

4 threads took 6.01583409309 seconds.


We can also use the threadpool

In [17]:
from multiprocessing.dummy import Pool as ThreadPool

In [18]:
pool = ThreadPool(4)
start = time.time()
results = pool.map(squared, x)
print("4 thread pool took {} seconds".format(time.time() - start))

4 thread pool took 4.84233379364 seconds


## I/O Bound Threading

open up some webpages

In [26]:
import requests

In [23]:
webpages = ["http://lifehacker.com/", "https://uoregon.edu/", "http://www.goducks.com/", "https://docs.python.org/2/howto/urllib2.html"]

In [27]:
def visit(url):
    return requests.get(url)

### Without threads

In [28]:
start = time.time()
for url in webpages:
    visit(url)
print("Serial method took {} second to open 4 webpages.".format(time.time() - start))

Serial method took 1.22712302208 second to open 4 webpages.


### With threads

In [29]:
t1 = Thread(target=visit, args=(webpages[0],))
t2 = Thread(target=visit, args=(webpages[1],))
t3 = Thread(target=visit, args=(webpages[2],))
t4 = Thread(target=visit, args=(webpages[3],))

start = time.time()
t1.start()
t2.start()
t3.start()
t4.start()
t1.join()
t2.join()
t3.join()
t4.join()
print("4 thread pool took {} seconds to open 4 webpages.".format(time.time() - start))

4 thread pool took 0.374730110168 seconds to open 4 webpages.


Threads can make I/O bound programs significantly faster.

Note:  I noticed that the threading actually ran slower if I wasn't connected with an ethernet cable at home.

# Squaring with multiprocessing

Let's do the same squaring function we did before, but this time with multiprocessing

In [30]:
from multiprocessing import Process
import time

In [31]:
len(x)

10000000

According to the python docs, pool cannot be used in the interactive interpreter.

It seems that this extends to an ipython notebook.
<a href="https://docs.python.org/3/library/multiprocessing.html#using-a-pool-of-workers">python pool docs</a>

In [32]:
num_pros = 4

pro_list = []
x3 = [10]*(len_x//num_pros)
for p in range(num_pros):
    p = Process(target=squareList, args=(x3,))
    pro_list.append(p)

start = time.time()
for p in pro_list:
    p.start()

for p in pro_list:
    p.join()

squareprocesstime = time.time() - start
print("Squaring 10 million numbers took {} seconds with 4 processes.".format(squareprocesstime))

Squaring 10 million numbers took 2.89520096779 seconds with 4 processes.


### Note:
When I ran this code on windows running these processes resulted in drastic speed decreases (30x).  But running the same code on ubuntu 15.10 works fine.  I'm still not sure why.  

Speed increase over serial method

In [33]:
squareprocesstime/serialprocesstime

0.5211550929723873

# Locks

When multiple processes try to access a single resource or variable, read and writes can overlap causing problems, so locks are necessary.

In [34]:
from multiprocessing import Lock, Value

In [35]:
lock = Lock()
counter = Value("i", 0)

def noLockCount():
    global counter
    for i in xrange(10000):
        counter.value += 1
    return counter

def lockCounter(lock):
    global counter
    
    with lock:
        for i in xrange(10000):
            counter.value += 1
    
    return counter

Here is analogous code for lockCounter

In [36]:
def sameLockCounter(lock):
    global counter
    
    lock.acquire()
    for i in xrange(10000):
        counter.value += 1
    lock.release()
    
    return counter

### No Lock

If we run 4 processes and add 1 to the counter 10,000 times each, we should expect to see 40,000 as our output

In [37]:
counter = Value("i", 0)
num_processes = 4
p_list = []

for i in range(num_processes):
    p = Process(target=noLockCount, args=())
    p_list.append(p)
    
for p in p_list:
    p.start()
    
for p in p_list:
    p.join()
    
print(counter.value)

12703


### With Lock

In [38]:
counter = Value("i", 0)
num_processes = 4
p_list = []

for i in range(num_processes):
    p = Process(target=lockCounter, args=(lock,))
    p_list.append(p)
    
for p in p_list:
    p.start()
    
for p in p_list:
    p.join()
    
print(counter.value)

40000


# Conclusiones

Just starting to use multiprocessing -> stick with threads unless you are cpu bound

-> use multiprocessing.dummy.ThreadPool.  It is the cleanest and easiest

# Otras opciones

- Clusters (reales y virtuales)
- Lambdas